# Lista 07 - Comparando Classificadores

# Exercício 01:

Analise o desempenho do kNN e de uma Regressão Logística Regularizada para **pelo menos um** dos conjuntos de dados abaixo:

* [Avaliação de carros](http://archive.ics.uci.edu/ml/datasets/Car+Evaluation)
* [Avaliação de vinhos](http://archive.ics.uci.edu/ml/datasets/Wine+Quality)
* [Resultados de partidas do jogo Dota](http://archive.ics.uci.edu/ml/datasets/Dota2+Games+Results) (desafiador!)

Para a questão, faça as seguintes tarefas:

* Realize treino, validação e teste
* Compare as métricas no teste
* Reporte a precisão, revocação, F1 e a matriz de confusão

Como já estamos no fim da matéria, você pode agora fazer uso da biblioteca scikit-learn. Afinal, no dia a dia, não implementamos tudo do zero. Abaixo temos os imports que vocês precisam. Leiam a API da biblioteca para saber como fazer uso da mesma.

In [5]:
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import StandardScaler

# Um fator importante é que o SKLearn não cria conjuntos de validação para você. Você tem algumas abordagens,
# uma é realizar um novo split no treino. Outra é fazer uso de classificadores com CV no fim.
# Tipo LogisticRegressionCV (ver na API). Por fim, você pode fazer uso da classe GridSearchCV.
# Leia a documentação da mesma.
from sklearn.model_selection import train_test_split

In [6]:
import pandas as pd
import numpy as ply

In [19]:
# YOUR CODE HERE
df = pd.read_csv("winequality-white.csv",sep=';')

#Logistic Reg train-test-valid
X_train, X_test, y_train, y_test = train_test_split(df.drop('quality',axis=1), df['quality'], test_size=0.30, random_state=101)

X_train, X_val, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=101)

scaler = StandardScaler()
X_new_train = scaler.fit_transform(X_train)

X_test = (X_test - X_train.mean()) / X_train.std(ddof=1)
X_test_new = scaler.transform(X_test)

#KNN train-test-valid
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(df.drop('quality',axis=1),df['quality'],test_size=0.30, random_state=101)

X_train_knn, X_valid_knn, y_train_knn, y_valid_knn = train_test_split(X_train_knn, y_train_knn, test_size=0.3, random_state=101)

logmodel = LogisticRegression()
logmodel.fit(X_new_train,y_train)
pred_log = logmodel.predict(X_test)

best = (0,0)
for nn in [2, 3, 4, 5, 6, 7, 8, 9, 10, 20]:

    knn = KNeighborsClassifier(n_neighbors=nn)
    knn.fit(X_train_knn, y_train_knn)
    pred_knn = knn.predict(X_valid_knn)

    accuracy = accuracy_score(y_valid_knn, pred_knn)
    if accuracy > best[0]:
        best = (accuracy, nn)

        
knn = KNeighborsClassifier(n_neighbors=best[1])
knn.fit(X_train_knn, y_train_knn)
pred_knn = knn.predict(X_test_knn)

print("Confusion Matrix - Logistic Regression")
print(confusion_matrix(y_test,pred_log))

print("Scores - Logistic Regression")
print(classification_report(y_test,pred_log))

print("Confusion Matrix - KNN = " + str(nn))
print(confusion_matrix(y_test,pred_knn))

print("Scores - KNN = " + str(nn))
print(classification_report(y_test, pred_knn))

#raise NotImplementedError()

Confusion Matrix - Logistic Regression
[[  0   0   3   2   0   0   0]
 [  0   0  40  15   0   0   0]
 [  0   0 239 197   2   0   0]
 [  0   2 121 502  41   0   0]
 [  0   0  16 193  42   0   0]
 [  0   0   0  43  11   0   0]
 [  0   0   0   0   1   0   0]]
Scores - Logistic Regression
             precision    recall  f1-score   support

          3       0.00      0.00      0.00         5
          4       0.00      0.00      0.00        55
          5       0.57      0.55      0.56       438
          6       0.53      0.75      0.62       666
          7       0.43      0.17      0.24       251
          8       0.00      0.00      0.00        54
          9       0.00      0.00      0.00         1

avg / total       0.48      0.53      0.49      1470

Confusion Matrix - KNN = 20
[[  0   0   3   2   0   0   0]
 [  0   1  34  16   3   1   0]
 [  0   4 205 189  34   6   0]
 [  0   8 186 389  73  10   0]
 [  0   0  39 165  47   0   0]
 [  0   1   7  30  11   5   0]
 [  0   1   0   0   

/home/gabrielperson/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Explique e discuta sobre os resultados encontrados no campo abaixo.

YOUR ANSWER HERE

Usando a regressão logistica tivemos uma quantidade considerável de predições, porém longe de modelos de regressão linear, tendo precisão abaixo de 0.5, o que é de se esperar quando e não aplicamos redução de dimensionalidade e podemos ter dados ruidosos.
O mesmo se observa utilizando KNN com K = 2, todas as metricas de qualificação do modelo tiveram valores abaixo de 0.5 e muitas predições incorretas, ao se observar a matriz de confusão. Certamente podem ser testados valores diferentes de K ou tambem outros modelos de classificação para se tentar uma predição melhor.